# Himmelblau Benchmark
## Imports

In [3]:
from bofire.benchmarks.single import Hartmann
from bofire.runners.api import run
from bofire.data_models.strategies.api import SoboStrategy, PolytopeSampler
from bofire.data_models.acquisition_functions.api import qEI
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain
from functools import partial
import pandas as pd
import os

SMOKE_TEST = os.environ.get("SMOKE_TEST")

C:\Users\shafeib\prj\bofire\bofire\utils\cheminformatics.py:19: UserWarning: mordred not installed. Mordred molecular descriptors cannot be used.
  warnings.warn(
C:\Users\shafeib\prj\bofire\bofire\surrogates\xgb.py:12: UserWarning: xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.
  warnings.warn("xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.")


## Random Optimization

In [5]:
def sample(domain):
    datamodel = PolytopeSampler(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled

def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()

random_results = run(
    Hartmann(dim=6, allowed_k=4),
    strategy_factory=lambda domain: strategies.map(PolytopeSampler(domain=domain)),
    n_iterations= 50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best -1.065: 100%|█████████████████████████████████████████████████| 50/50 [00:02<00:00, 21.40it/s]


## SOBO (GPEI) Optimization

In [ ]:
from functools import partial

def strategy_factory(domain: Domain):
    data_model = SoboStrategy(domain=domain, acquisition_function=qEI(), num_raw_samples=512, num_restarts=24)
    return strategies.map(data_model)

bo_results = run(
    Hartmann(dim=6, allowed_k=4),
    strategy_factory=strategy_factory,
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best -3.034:  30%|██████████████▋                                  | 15/50 [06:04<13:05, 22.45s/it]